In [1]:
import sys
from datetime import datetime

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1639197913732_0006,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession\
    .builder\
    .appName("SolarDataBilling3")\
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark.catalog.setCurrentDatabase("solarData")
dfContracts = spark.sql("select * from contracts")
dfContracts.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------------------+------+------+-------+
|        clientname|           startdate|             enddate|rate10|rate50|rate100|
+------------------+--------------------+--------------------+------+------+-------+
|       Apple, Inc.|2021-01-01T00:00:...|2021-12-31T23:59:...|  0.01|  0.03|   0.05|
|   Microsoft, Inc.|2021-01-01T00:00:...|2021-12-31T23:59:...| 0.008| 0.015|  0.049|
|Tesla Motors, Inc.|2021-01-01T00:00:...|2021-12-31T23:59:...|  0.01|  0.02|  0.054|
+------------------+--------------------+--------------------+------+------+-------+

In [7]:
strQuery = f"SELECT year, month, SUM(watts) AS total_watts, speed FROM solar_data GROUP BY year, month, speed ORDER BY month"
dfSolarData = spark.sql(strQuery)
dfSolarData.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+-----------+-----+
|year|month|total_watts|speed|
+----+-----+-----------+-----+
|2021|    1|       1170|   50|
|2021|    1|       5635|  100|
|2021|    2|       4393|  100|
|2021|    2|        213|   10|
|2021|    3|        302|   10|
|2021|    3|       3984|  100|
|2021|    4|        360|   10|
|2021|    4|       4438|  100|
|2021|    5|       5141|  100|
|2021|    6|       5147|  100|
|2021|    7|       2975|  100|
|2021|    8|       4513|  100|
|2021|    8|        762|   10|
|2021|    9|        417|   50|
|2021|    9|        775|   10|
|2021|    9|       1259|  100|
|2021|   10|       2612|  100|
|2021|   11|        231|   50|
|2021|   11|        702|   10|
|2021|   11|       2758|  100|
+----+-----+-----------+-----+
only showing top 20 rows

In [11]:
solarDataCollect = dfSolarData.collect()
contractsCollect = dfContracts.collect()

billsToCreate = []

for contract in contractsCollect:
    
    newClient = {
        "clientName": contract.clientname,
        "total" : 0.00,
        "details" : []
    }
    
    for solarData in solarDataCollect:
        datetime_object = datetime.strptime(f"{solarData.month}", "%m")
        monthName = datetime_object.strftime("%B")
        
        rate = 0.00
        if solarData.speed == 100:
            rate = contract.rate100
        elif solarData.speed == 50:
            rate = contract.rate50
        elif solarData.speed == 10:
            rate = contract.rate10
        
        
        detailTotal = rate * solarData.total_watts
        
        detail = {
            "period" : f"{monthName} {solarData.year}",
            "speed" : solarData.speed,
            "watts" : solarData.total_watts,
            "rate" : rate,
            "total" : float(format(detailTotal, ".2f"))
        }
        
        newClient['details'].append(detail)
        newClient['total'] += detail['total']
        
    print(newClient)
        
    billsToCreate.append(newClient)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'clientName': 'Apple, Inc.', 'total': 2441.56, 'details': [{'period': 'January 2021', 'speed': 100, 'watts': 5635, 'rate': 0.05, 'total': 281.75}, {'period': 'January 2021', 'speed': 50, 'watts': 1170, 'rate': 0.03, 'total': 35.1}, {'period': 'February 2021', 'speed': 10, 'watts': 213, 'rate': 0.01, 'total': 2.13}, {'period': 'February 2021', 'speed': 100, 'watts': 4393, 'rate': 0.05, 'total': 219.65}, {'period': 'March 2021', 'speed': 10, 'watts': 302, 'rate': 0.01, 'total': 3.02}, {'period': 'March 2021', 'speed': 100, 'watts': 3984, 'rate': 0.05, 'total': 199.2}, {'period': 'April 2021', 'speed': 100, 'watts': 4438, 'rate': 0.05, 'total': 221.9}, {'period': 'April 2021', 'speed': 10, 'watts': 360, 'rate': 0.01, 'total': 3.6}, {'period': 'May 2021', 'speed': 100, 'watts': 5141, 'rate': 0.05, 'total': 257.05}, {'period': 'June 2021', 'speed': 100, 'watts': 5147, 'rate': 0.05, 'total': 257.35}, {'period': 'July 2021', 'speed': 100, 'watts': 2975, 'rate': 0.05, 'total': 148.75}, {'peri

In [12]:
for billToCreate in billsToCreate:
    df = spark.createDataFrame(billsToCreate)
    df.show()
    df.write.format("json").mode("overwrite").save("s3://santos-solar-data/bills/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+-------+
|        clientName|             details|  total|
+------------------+--------------------+-------+
|       Apple, Inc.|[[total -> 281.75...|2441.56|
|   Microsoft, Inc.|[[total -> 276.12...|2352.34|
|Tesla Motors, Inc.|[[total -> 304.29...|2608.02|
+------------------+--------------------+-------+

+------------------+--------------------+-------+
|        clientName|             details|  total|
+------------------+--------------------+-------+
|       Apple, Inc.|[[total -> 281.75...|2441.56|
|   Microsoft, Inc.|[[total -> 276.12...|2352.34|
|Tesla Motors, Inc.|[[total -> 304.29...|2608.02|
+------------------+--------------------+-------+

+------------------+--------------------+-------+
|        clientName|             details|  total|
+------------------+--------------------+-------+
|       Apple, Inc.|[[total -> 281.75...|2441.56|
|   Microsoft, Inc.|[[total -> 276.12...|2352.34|
|Tesla Motors, Inc.|[[total -> 304.29...|2608.02